
# About Dataset

### Context This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

Content 5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset. Columns

    asin - ID of the product, like B000FA64PK
    helpful - helpfulness rating of the review - example: 2/3.
    overall - rating of the product.
    reviewText - text of the review (heading).
    reviewTime - time of the review (raw).
    reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
    reviewerName - name of the reviewer.
    summary - summary of the review (description).
    unixReviewTime - unix timestamp.

Acknowledgements This dataset is taken from Amazon product data, Julian McAuley, UCSD website. http://jmcauley.ucsd.edu/data/amazon/

License to the data files belong to them.

Inspiration

    Sentiment analysis on reviews.
    Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.
    Fake reviews/ outliers.
    Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).
    Any other interesting analysis

Best Practises

   1. Preprocessing And Cleaning
   2. Train Test Split
   3. BOW,TFIDF,Word2vec(Vectorization)
   4. Train ML algorithms



In [10]:
##Load dataset
import pandas as pd
df = pd.read_csv('all_kindle_review.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [11]:
##Data ko review text aur rating 2 columns me baanto baki columns ni chahiye uske
df = df[['reviewText','rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [12]:
df.shape

(12000, 2)

In [13]:
##missing values
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [14]:
df['rating'].unique()
##Yani 5 tarah ki unique ratings hai 

array([3, 5, 4, 2, 1], dtype=int64)

In [15]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

## Preprocessing and cleaning

In [16]:
##positive review should be marked 1 and negative with 0
df['rating'] = df['rating'].apply(lambda x:0 if x<3 else 1)

In [20]:
df['rating'].unique()
##ab sirf 1 and 0 hai bache 

array([1, 0], dtype=int64)

In [23]:
df['rating'].value_counts()
##Yani 8000 positve review aur 4000 negtive

rating
1    8000
0    4000
Name: count, dtype: int64

In [24]:
## 1.Lower all the cases
df['reviewText'] = df['reviewText'].str.lower()

In [25]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [26]:
## Removing special chars and stopswords
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ramsa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
#used to remove html tags
from bs4 import BeautifulSoup

In [28]:
## Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url 
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))

C:\Users\ramsa\AppData\Local\Temp\ipykernel_10772\3736800740.py:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())


In [29]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [30]:
## Lemmatizer
from nltk.stem import WordNetLemmatizer

In [31]:
lemmatizer = WordNetLemmatizer()

In [33]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [34]:
df['reviewText'] = df['reviewText'].apply(lambda x : lemmatize_words(x))

In [35]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [36]:
##Train test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df['reviewText'],df['rating'],test_size=0.20)

## Vectorization

In [46]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()
X_train_bow = bow.fit_transform(X_train)
X_test_bow = bow.transform(X_test)

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [48]:
X_train_tfidf

<9600x35503 sparse matrix of type '<class 'numpy.float64'>'
	with 431790 stored elements in Compressed Sparse Row format>

In [51]:
##Apply ML ALGO
from sklearn.naive_bayes import MultinomialNB

nb_model_bow = MultinomialNB().fit(X_train_bow, y_train)
nb_model_tfidf = MultinomialNB().fit(X_train_tfidf, y_train)



In [53]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [54]:
y_pred_bow = nb_model_bow.predict(X_test_bow)

In [55]:
y_pred_tfidf = nb_model_bow.predict(X_test_tfidf)

In [56]:
confusion_matrix(y_test,y_pred_bow)

array([[ 566,  252],
       [ 142, 1440]], dtype=int64)

In [57]:
print("Bow Accuracy: ",accuracy_score(y_test,y_pred_bow))

Bow Accuracy:  0.8358333333333333


In [58]:
confusion_matrix(y_test,y_pred_tfidf)

array([[ 398,  420],
       [  73, 1509]], dtype=int64)

In [59]:
print("TFIDF accuracy: ",accuracy_score(y_test,y_pred_tfidf))

TFIDF accuracy:  0.7945833333333333
